In [26]:
import pandas as pd
import numpy as np
from pandasql import sqldf


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 30
pd.get_option("display.max_rows")
#pd.set_option('display.max_rows', 100)

import mysql.connector # pip install mysql-connector-python


import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

my_host = os.getenv("my_host")
my_user = os.getenv("my_user")
my_pswd = os.getenv("my_pswd")

os.listdir()

['.Rhistory',
 'requirements.txt',
 'mysql_basics.ipynb',
 'README.md',
 '.env',
 'mysql_sqlalchemy.ipynb',
 '.git']

### Connect to MYSQL server:
* https://www.datacamp.com/tutorial/tutorial-how-to-execute-sql-queries-in-r-and-python
* https://www.w3schools.com/mysql/default.asp

In [7]:
mydb = mysql.connector.connect(
    host= my_host
    ,user= my_user
    ,passwd= my_pswd
    
)

mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

mycursor.close()
mydb.close()

('db_rizal',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


### Connect to DB:

In [8]:
mydb = mysql.connector.connect(
        host= my_host
        ,user= my_user
        ,passwd= my_pswd
        ,database="db_rizal"
        )

mycursor = mydb.cursor()


mycursor.execute("SHOW tables")

for x in mycursor:
    print(x)

('employees',)
('employees_v2',)
('players',)
('studentinfo',)
('tbl_players',)
('tbl_players_v2',)
('tbl_test',)
('tmp_count',)
('tmp_freq',)


### DROP & CREATE NEW TABLE:
* primary key are index by default

In [9]:
mycursor.execute("""DROP TABLE studentinfo;""")

In [10]:
qry = """
       
       CREATE TABLE studentinfo
            (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name varchar(255) NOT NULL,
            subject varchar(255) NOT NULL
            )
        """

mycursor.execute(qry)

### INSERT INTO :

In [11]:
### insert 1 row:
qry = """
            INSERT INTO studentinfo (name, subject)
            VALUES ('Marry','Math' )
        """
mycursor.execute(qry)

In [12]:
### insert multiple rows:

qry = """
        INSERT INTO studentinfo (name, subject)
        VALUES 
        ("Abid", "Stat"),
        ("Carry", "Math"),
        ("Ali","Data Science"),
        ("Nisha","Data Science"),
        ("Matthew","Math"),
        ("Henry","Data Science")
      """

mycursor.execute(qry)

In [13]:
# verify data exist temporarily 
mycursor.execute("""SELECT * FROM studentinfo""")
mycursor.fetchall()

[(1, 'Marry', 'Math'),
 (2, 'Abid', 'Stat'),
 (3, 'Carry', 'Math'),
 (4, 'Ali', 'Data Science'),
 (5, 'Nisha', 'Data Science'),
 (6, 'Matthew', 'Math'),
 (7, 'Henry', 'Data Science')]

### CREATE UNIQUE INDEX:
* Creates a unique index on a table. Duplicate values are not allowed

In [14]:
qry = """
        CREATE UNIQUE INDEX idx_name
        ON studentinfo (name);
      """

mycursor.execute(qry)

### CREATE INDEX:
* Generally, it's more efficient to create indexes in MySQL after data insertion, especially for large datasets.

In [15]:
# on single column
qry = """ 
        CREATE INDEX idx_subject
        ON studentinfo (subject);
        """
mycursor.execute(qry)

In [16]:
# on combine columns
qry = """
        CREATE INDEX idx_name_subject
        ON studentinfo (name, subject);

"""
mycursor.execute(qry)

### Save data in DB:
* Without a commit, the data will be lost after the machine restarts.

In [17]:
#mydb.commit()

### Download tables from sql qry:

In [18]:
df = pd.read_sql(""" SELECT * FROM  studentinfo""",mydb)

print(df.shape)
df

(7, 3)


/var/folders/_q/ktfjns2n4_l9c5_kj8k08w540000gn/T/ipykernel_6477/2277355563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(""" SELECT * FROM  studentinfo""",mydb)


,id,name,subject
0,2,Abid,Stat
1,4,Ali,Data Science
2,3,Carry,Math
3,7,Henry,Data Science
4,1,Marry,Math
5,6,Matthew,Math
6,5,Nisha,Data Science


In [19]:
table_name = "studentinfo"

mycursor.execute(f"SELECT * FROM {table_name} ORDER BY id")

#print(mycursor.description)

df = pd.DataFrame(mycursor.fetchall()
                  ,columns=[i[0] for i in mycursor.description]
                 )


print(df.shape)
df

(7, 3)


,id,name,subject
0,1,Marry,Math
1,2,Abid,Stat
2,3,Carry,Math
3,4,Ali,Data Science
4,5,Nisha,Data Science
5,6,Matthew,Math
6,7,Henry,Data Science


In [20]:
sqldf("""SELECT 
        subject
        ,COUNT(1) AS FREQ
        FROM df
        GROUP BY subject
        """,locals())

,subject,FREQ
0,Data Science,3
1,Math,3
2,Stat,1


In [21]:
qry = """ 
        SELECT 
        subject
        ,COUNT(1) AS FREQ
        FROM studentinfo
        GROUP BY subject
       """

mycursor.execute(qry)
mycursor.fetchall()

[('Data Science', 3), ('Math', 3), ('Stat', 1)]

In [22]:
mycursor.execute("""DROP TABLE IF EXISTS tmp_freq ; """)


mycursor.execute("SHOW tables")

for x in mycursor:
    print(x)

('employees',)
('employees_v2',)
('players',)
('studentinfo',)
('tbl_players',)
('tbl_players_v2',)
('tbl_test',)
('tmp_count',)


In [23]:
qry = """ 
    CREATE TABLE tmp_freq AS
            SELECT 
            subject
            ,COUNT(1) AS FREQ
            FROM studentinfo
            GROUP BY subject
    """
mycursor.execute(qry)
#mydb.commit()

In [24]:
mycursor.execute(""" SELECT * FROM tmp_freq """)
mycursor.fetchall()

[('Data Science', 3), ('Math', 3), ('Stat', 1)]

In [25]:
mycursor.close()
mydb.close()